<a href="https://colab.research.google.com/github/Erickrus/llm/blob/main/beam_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Greedy Search Decoder and Beam Search Decoder implementation

https://machinelearningmastery.com/beam-search-decoder-natural-language-processing/

### Greedy Search Decoder

A simple approximation is to use a greedy search that selects the most likely word at each step in the output sequence.

This approach has the benefit that it is very fast, but the quality of the final output sequences may be far from optimal.

We can demonstrate the greedy search approach to decoding with a small contrived example in Python.

We can start off with a prediction problem that involves a sequence of 10 words. Each word is predicted as a probability distribution over a vocabulary of 5 words.

We will assume that the words have been integer encoded, such that the column index can be used to look-up the associated word in the vocabulary. Therefore, the task of decoding becomes the task of selecting a sequence of integers from the probability distributions.

The argmax() mathematical function can be used to select the index of an array that has the largest value. We can use this function to select the word index that is most likely at each step in the sequence. This function is provided directly in numpy.

The greedy_decoder() function below implements this decoder strategy using the argmax function.

In [ ]:
#@title greedy_decoder()
import numpy as np

# greedy decoder
def greedy_decoder(time_steps):
    sequence = []
    # index for largest probability each row
    for step_id, logits in enumerate(time_steps):
        sequence.append(greedy_decode(logits))
        print(f"step {step_id}:", sequence)
    return sequence

def greedy_decode(logits):
    return np.argmax(logits)

# define a sequence of 10 words over a vocab of 5 words
data = [
    [0.1, 0.2, 0.3, 0.4, 0.5],
    [0.5, 0.4, 0.3, 0.2, 0.1],
    [0.1, 0.2, 0.3, 0.4, 0.5],
    [0.5, 0.4, 0.3, 0.2, 0.1],
    [0.1, 0.2, 0.3, 0.4, 0.5],
    [0.5, 0.4, 0.3, 0.2, 0.1],
    [0.1, 0.2, 0.3, 0.4, 0.5],
    [0.5, 0.4, 0.3, 0.2, 0.1],
    [0.1, 0.2, 0.3, 0.4, 0.5],
    [0.5, 0.4, 0.3, 0.2, 0.1]]
data = np.array(data)
# decode sequence
result = greedy_decoder(data)

step 0: [4]
step 1: [4, 0]
step 2: [4, 0, 4]
step 3: [4, 0, 4, 0]
step 4: [4, 0, 4, 0, 4]
step 5: [4, 0, 4, 0, 4, 0]
step 6: [4, 0, 4, 0, 4, 0, 4]
step 7: [4, 0, 4, 0, 4, 0, 4, 0]
step 8: [4, 0, 4, 0, 4, 0, 4, 0, 4]
step 9: [4, 0, 4, 0, 4, 0, 4, 0, 4, 0]


### Beam Search Decoder

Another popular heuristic is the beam search that expands upon the greedy search and returns a list of most likely output sequences.

Instead of greedily choosing the most likely next step as the sequence is constructed, the beam search expands all possible next steps and keeps the k most likely, where k is a user-specified parameter and controls the number of beams or parallel searches through the sequence of probabilities.

The local beam search algorithm keeps track of k states rather than just one. It begins with k randomly generated states. At each step, all the successors of all k states are generated. If any one is a goal, the algorithm halts. Otherwise, it selects the k best successors from the complete list and repeats.

— Pages 125-126, Artificial Intelligence: A Modern Approach (3rd Edition), 2009.


We can define a function to perform the beam search for a given sequence of probabilities and beam width parameter k. At each step, each candidate sequence is expanded with all possible next steps. Each candidate step is scored by multiplying the probabilities together. The k sequences with the most likely probabilities are selected and all other candidates are pruned. The process then repeats until the end of the sequence.

Probabilities are small numbers and multiplying small numbers together creates very small numbers. To avoid underflowing the floating point numbers, the natural logarithm of the probabilities are added together, which keeps the numbers larger and manageable. Further, it is also common to perform the search by minimizing the score. This final tweak means that we can sort all candidate sequences in ascending order by their score and select the first k as the most likely candidate sequences.

The beam_search_decoder() function below implements the beam search decoder.

In [ ]:
#@title beam_search_decoder()
import math
import numpy as np

def beam_search_decoder(time_steps, beta):
    # sequence maintain a data structure of [[sequence list], score]
    sequences = [[list(), 0.0]]
    # walk over each step in sequence
    for step_id, logits in enumerate(time_steps):
        sequences = beam_search(sequences, logits, beta, step_id)
    return sequences

# beam search
def beam_search(sequences, logits, beta, step_id=0):
    all_candidates = list()
    # expand each current candidate
    for i in range(len(sequences)):
        seq, score = sequences[i]
        for j in range(len(logits)):
            candidate = [seq + [j], score - math.log(logits[j])]
            all_candidates.append(candidate)
    # order all candidates by score
    ordered = sorted(all_candidates, key=lambda tup:tup[1])
    # trim β best
    sequences = ordered[:beta]
    print(f"step {step_id}:",)
    for seq in sequences:
        print(seq)
    return sequences

# define a sequence of 10 words over a vocab of 5 words
data = [[0.1, 0.2, 0.3, 0.4, 0.5],
    [0.5, 0.4, 0.3, 0.2, 0.1],
    [0.1, 0.2, 0.3, 0.4, 0.5],
    [0.5, 0.4, 0.3, 0.2, 0.1],
    [0.1, 0.2, 0.3, 0.4, 0.5],
    [0.5, 0.4, 0.3, 0.2, 0.1],
    [0.1, 0.2, 0.3, 0.4, 0.5],
    [0.5, 0.4, 0.3, 0.2, 0.1],
    [0.1, 0.2, 0.3, 0.4, 0.5],
    [0.5, 0.4, 0.3, 0.2, 0.1]]
data = np.array(data)
# decode sequence
result = beam_search_decoder(data, beta=3)

step 0:
[[4], 0.6931471805599453]
[[3], 0.916290731874155]
[[2], 1.2039728043259361]
step 1:
[[4, 0], 1.3862943611198906]
[[4, 1], 1.6094379124341003]
[[3, 0], 1.6094379124341003]
step 2:
[[4, 0, 4], 2.0794415416798357]
[[4, 0, 3], 2.3025850929940455]
[[4, 1, 4], 2.3025850929940455]
step 3:
[[4, 0, 4, 0], 2.772588722239781]
[[4, 0, 4, 1], 2.995732273553991]
[[4, 0, 3, 0], 2.995732273553991]
step 4:
[[4, 0, 4, 0, 4], 3.4657359027997265]
[[4, 0, 4, 0, 3], 3.6888794541139363]
[[4, 0, 4, 1, 4], 3.6888794541139363]
step 5:
[[4, 0, 4, 0, 4, 0], 4.1588830833596715]
[[4, 0, 4, 0, 4, 1], 4.382026634673881]
[[4, 0, 4, 0, 3, 0], 4.382026634673881]
step 6:
[[4, 0, 4, 0, 4, 0, 4], 4.852030263919617]
[[4, 0, 4, 0, 4, 0, 3], 5.075173815233827]
[[4, 0, 4, 0, 4, 1, 4], 5.075173815233827]
step 7:
[[4, 0, 4, 0, 4, 0, 4, 0], 5.545177444479562]
[[4, 0, 4, 0, 4, 0, 4, 1], 5.768320995793772]
[[4, 0, 4, 0, 4, 0, 3, 0], 5.768320995793772]
step 8:
[[4, 0, 4, 0, 4, 0, 4, 0, 4], 6.238324625039508]
[[4, 0, 4, 0, 4

### Contrastive Search: Theory and Implementation
Theory

Contrastive search is a method for generating sequences (e.g., text) that balances diversity and quality. This approach aims to avoid repetitive and degenerate text generation by penalizing less diverse sequences. Here are the
key elements:

- Candidate Recall: At each step, generate a set of candidate tokens using the model's logits.
- Degeneration Penalty: Apply a penalty to the candidate tokens to prevent repetitive patterns.
- Re-ranking: Re-rank the candidates based on the model's confidence (probability) and the degeneration penalty.


#### Key Parameters:

- top_k: The number of top candidates considered at each step.
- penalty_alpha: The weight of the degeneration penalty. A higher value discourages repetitive sequences more strongly.


#### Implementation
The provided code implements contrastive search in TensorFlow for text generation using a Transformer-based model. Here’s a detailed breakdown of the process:

##### Initialization:

Various configurations like logits_processor, logits_warper, max_length, pad_token_id, eos_token_id, etc., are initialized.
Initialize tensors for storing scores, attention weights, hidden states, and the generated sequence.

##### First Step Preparation:

Encode the initial input sequence to get the past key values (cached states), logits for the next step, and hidden states.
These are used for the first contrastive search step.

##### Contrastive Search Loop:

Candidate Recall:
Process logits using logits_processor and logits_warper.
Apply softmax to get probabilities and select top-k candidates.
Re-ranking with Degeneration Penalty:
Compute cosine similarity between the hidden states of the last generated token and the candidate tokens.
Apply the degeneration penalty and re-rank the candidates.
Selection and Update:
Select the best candidate based on the re-ranked scores.
Update the sequence, hidden states, and cached values for the next step.
Termination Check:
Check if the end-of-sequence token is generated or the maximum length is reached.

##### Output Preparation:

Collect and return the generated sequence along with optional scores, attentions, and hidden states if requested.

#### Key Functions and Code Explanation
gather_best_candidate:

Gathers slices indexed by selected_idx_stacked from a nested structure of tensors.
This is used to select the best candidate sequences at each step.
contrastive_search_cond_fn:

Checks whether the generation process should continue based on unfinished sequences.
contrastive_search_body_fn:

The core function implementing the contrastive search logic.
Handles candidate recall, re-ranking, and updates the sequence, hidden states, and model cache.
Main Logic Flow
First Step:

Prepare inputs and encode the initial prefix.
Obtain past key values, logits, and hidden states.
Process logits and get top-k candidates.
Compute the degeneration penalty and select the best candidate.
Update the sequence and prepare for the next step.
Subsequent Steps:

For each step, recall top-k candidates.
Apply degeneration penalty and re-rank.
Select the best candidate and update the sequence.
Continue until the end-of-sequence token is generated or the maximum length is reached.
Final Output:

Return the generated sequence, optionally with scores, attentions, and hidden states.
Example Usage
Here’s an example usage provided in the docstring:

python
Copy code
from transformers import AutoTokenizer, TFAutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("facebook/opt-125m")
model = TFAutoModelForCausalLM.from_pretrained("facebook/opt-125m")
# Set pad_token_id to eos_token_id because OPT does not have a PAD token
model.config.pad_token_id = model.config.eos_token_id
input_prompt = "DeepMind Company is"
input_ids = tokenizer(input_prompt, return_tensors="tf")
outputs = model.contrastive_search(input_ids, penalty_alpha=0.6, top_k=4, max_length=64)
generated_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print(generated_text)
This example initializes the model and tokenizer, sets the input prompt, and performs contrastive search to generate a sequence. The result is decoded and printed.

Summary
Contrastive search is a powerful method to generate high-quality and diverse text sequences. The implementation leverages top-k candidate recall and degeneration penalties to balance between diversity and coherence. This method can be applied to various sequence generation tasks, including text-to-text, speech-to-text, and vision-to-text models.

https://medium.com/axinc-ai/contrastive-search-a-token-selection-algorithm-that-can-suppress-repetition-in-language-10712d5408da